import tensorflow_hub as hub
model = hub.load("model")
def get_embedding(s):
    embedding=model([s])
    return embedding.numpy()[0]


In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")
#es.indices.delete(index='documents', ignore=[400, 404])
mapping={"settings":{"number_of_shards": 1,"number_of_replicas": 0},
"mappings": {"dynamic": "true","_source": {"enabled": "true"},
"properties": {
"title": {
"type": "text"
},
"Doc_vector": {
"type": "dense_vector",
"dims": 384
},
"date":
    {
        "type": "date"
    },
"category":
    {
        "type":"keyword"
    }

}}}
es.indices.create(
    index="documents",
    body=mapping,
    ignore=400
)

es.indices.delete(index='documents', ignore=[400, 404])
mapping={"settings":{"number_of_shards": 1,"number_of_replicas": 0},
"mappings": {"dynamic": "true","_source": {"enabled": "true"},
"properties": {
"title": {
"type": "text"
},
"Doc_vector": {
"type": "text",

},
"date":
    {
        "type": "date"
    },
"category":
    {
        "type":"keyword"
    }

}}}
es.indices.create(
    index="documents",
    body=mapping,
    ignore=400 # ignore 400 already exists code
)

data=list(i.replace("\n","") for i in open("news-dump_2020_1.txt",encoding='utf-8'))
i=0
l=len(data)
res=[]
while i<l:
    
    if (len(data[i])!=11) or (data[i].count("/")!=3):
        i+=1
    else:
        date = data[i]
        zz=data[i+1].index("$")
        category, title = data[i+1][:zz],data[i+1][zz+1:]
        i+=2
        article=""
        while i<l and (not((len(data[i])==11) and (data[i].count("/")==3))):
            article+=data[i]+" "
            i+=1
        res.append((date,category,title,article))
    
    #if i>100:break

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('./paraphrase-MiniLM-L6-v2')


In [20]:
import pandas as pd
data=pd.read_csv("news-filter-data.csv")
import numpy as np

encodings= np.load("encodings.npy")

In [41]:
import pickle
fff=open("filter-content-list",'rb')
zzz=pickle.load(fff)
fff.close()

In [43]:

for i in range(30):
    print([zzz[i]])

['Kundapur-Karwar NH66 stretch to be widened\nNew Year appears to have brought new hopes to the people of coastal Karnataka with the National Highways Authority of India (NHAI) initiating steps to widen a 189-km stretch of the national highway between Kundapura and Karwar (up to Goa border).This NH 66 stretch is a part of the 296-km highway from Goa border (near Karwar) to Kerala border (near Talapady) in the State.The stretch between Kundapur and Talapady, via Mangalore, on the same highway is now being widened by the NHAI.With the new project on the anvil, the highway passing through Dakshina Kannada, Udupi, and Uttara Kannada districts via Karwar, Ankola, Kumta, Honnavar, Bhatkal, Kundapur, Udupi, Surathkal, Mangalore, Thokkottu, and Talapady is all set to get a facelift. Sources in the NHAI told\nThe Hindu that it had called for Request for Qualification (RFQ) from companies interested in taking up the Kundapura-Goa stretch.\nSo far, 39 companies had responded.The NHAI had proposed

In [53]:
from ast import literal_eval
res=[]
for i in range(len(encodings)):
    res.append(tuple(data[j][i] if j=="date" else " ".join(literal_eval(data[j][i])) for j in ("date","category","headline")))
#res=[(a,b,c) for a,b,c in res if b in ('frontpag','sport','busi','nation','intern')]
mapping={'frontpag':'frontpage','sport':'sport','busi':'business','nation':'national','intern':'international'}
qqq=[]
new_res=[]
cc=0
for a,b,c in res:
    if b in mapping:
        ind=zzz[cc].index("\n")
        titl = zzz[cc][:ind]
        doccc = zzz[cc][ind+1:]
        
        qqq.append((a,titl,doccc))
        b=mapping[b]
        new_res.append((a,b,c,encodings[cc]))
    cc+=1
print(len(new_res),len(zzz))
res=new_res

111877 111887


111877

In [55]:
 with open('documents.pkl', 'wb') as f:
...   pickle.dump(qqq, f)



In [5]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")
indd=27001
for date,category,title,doc in res[27000:]:
    date=date.replace("/","-")[:-1]
    #doc=get_embedding(doc)
    es.index(index="documents" , id=indd, document={"Doc_vector":doc,"date":date,"title":title,"category":category})
    print(indd,end='\r')
    indd+=1

In [40]:
fff=open("documents.pkl",'rb')
zzz=pickle.load(fff)
print(zzz[0])
print(len(zzz))

('2012/01/01/', 'Kundapur-Karwar NH66 stretch to be widened', 'New Year appears to have brought new hopes to the people of coastal Karnataka with the National Highways Authority of India (NHAI) initiating steps to widen a 189-km stretch of the national highway between Kundapura and Karwar (up to Goa border).This NH 66 stretch is a part of the 296-km highway from Goa border (near Karwar) to Kerala border (near Talapady) in the State.The stretch between Kundapur and Talapady, via Mangalore, on the same highway is now being widened by the NHAI.With the new project on the anvil, the highway passing through Dakshina Kannada, Udupi, and Uttara Kannada districts via Karwar, Ankola, Kumta, Honnavar, Bhatkal, Kundapur, Udupi, Surathkal, Mangalore, Thokkottu, and Talapady is all set to get a facelift. Sources in the NHAI told\nThe Hindu that it had called for Request for Qualification (RFQ) from companies interested in taking up the Kundapura-Goa stretch.\nSo far, 39 companies had responded.The 

In [39]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")

for i in range(1,30):
    query =  {"bool":{"filter":{"term":{"_id":i}}}}
    print([es.search(index="documents",body={'size': 1,'query': query})['hits']['hits'][0]['_source']['title']],[zzz[i-1][1]])
    

C:\Users\ivnag\AppData\Local\Temp/ipykernel_2008/1640841572.py:6: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  print([es.search(index="documents",body={'size': 1,'query': query})['hits']['hits'][0]['_source']['title']],[zzz[i-1][1]])


['kundapur karwar nh66 stretch widen'] ['Kundapur-Karwar NH66 stretch to be widened']
['peopl spar power cut summer'] ['a']
['“govern win accept amendments”'] ['People may be spared of power cuts during summer']
['manmohan new year pledg ‘honest effici rule'] ['a']
['karunanidhi disagre chandi view mullaperiyar'] ["“Government won't accept all amendments”"]
['lokpal bill remain aliv till end current lok sabha term'] ['a']
['fishermen recal ‘thane horror'] ["Manmohan's New Year pledge: ‘honest and more efficient' rule"]
['more civilian die syrian crackdown'] ['a']
['cat reject petit select air chief'] ["Karunanidhi disagrees with Chandy's views on Mullaperiyar"]
['rain lash southern district'] ['a']
['nine die consum hooch'] ['Lokpal Bill will remain alive till end of current Lok Sabha term']
['metro man call day'] ['a']
['sri lanka scrap visa arriv'] ["Fishermen recall ‘Thane' horror"]
['syrian opposit unit'] ['a']
['ramakrishnan knight'] ['More civilians die in Syrian crackdown']
['‘i